In [ ]:
#Run docker container
sudo docker run -d --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''

#Exec docker container
docker exec -it sparkbook

#Run Spark
#go on browser
localhost:8881

In [13]:
#Run1
%matplotlib inline

import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.style.use('ggplot')

In [ ]:
#Run2
import pyspark as ps

spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('Instacart')
         .getOrCreate()
        )

sc = spark.sparkContext
sc

In [14]:
paris_listings = pd.read_csv('Data/Paris/listings.csv', nrows=50000)

/Users/winrichsy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
#drop useless columns
useless_columns = ['square_feet','country','listing_url','scrape_id','last_scraped','experiences_offered', 'license', 
                   'xl_picture_url','host_url','host_name','host_thumbnail_url','street','host_listings_count',
                   'neighbourhood_group_cleansed','state','market', 'calendar_last_scraped','host_picture_url',
                   'host_acceptance_rate', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
                   'maximum_maximum_nights', 'jurisdiction_names', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms','maximum_nights_avg_ntm',
                   'has_availability','country_code','host_acceptance_rate','thumbnail_url','medium_url','weekly_price','monthly_price',
                   'calendar_updated','is_business_travel_ready', 'interaction', 'interaction', 'house_rules']
paris_data_shortened = paris_listings.drop(columns = useless_columns)

In [29]:
#save new csv
singapore = 'Data/Singapore/'
london = 'Data/London/'
paris = 'Data/Paris/'

paris_data_shortened.to_csv (paris+'shortened_listings.csv', index = None, header=True)

In [ ]:
#Run3
paris_listings = spark.read.csv(paris +'shortened_listings.csv',
                           header = True,
                           sep = ",",
                           inferSchema = True)

paris_calendar = spark.read.csv(paris+'shortened_calendar.csv',
                            header = True,
                            sep = ",",
                            inferSchema = True)

paris_reviews = spark.read.csv(paris+'shortened_reviews.csv',
                                     header = True,
                                     sep = ",",
                                     inferSchema = True)

#===========================

In [ ]:
paris_calendar